<a href="https://colab.research.google.com/github/2025-02-FML-team/WV-Team/blob/main/notebooks/01_data_bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
import os 
import pandas as pd
import requests
from urllib.parse import urlparse
from pathlib import Path

data_dir = Path(os.path.join(os.getcwd(), "../data/")).resolve()
data_dir

PosixPath('/Volumes/Backup/Workspace/ML/WV-Team/data')

In [89]:
csv_url = data_dir / 'whisky_full.csv'

In [90]:
csv_url

PosixPath('/Volumes/Backup/Workspace/ML/WV-Team/data/whisky_full.csv')

In [91]:
# read the CSV file located in data/whiskies.csv
df = pd.read_csv(csv_url)

df = df.drop(['name_eng-2', 'category_specific'], axis=1)

# show the first few rows
df.head()

,id,name,name_eng,category,volume,alcohol,price,brand,nation,bottled,region,region_detail,distillery,importer,external_image_url,image_url,full_image_url
0,63800001,1423 S.B.S 가이아나 2020,1423 S.B.S Guyana 2020,기타,700,66.4,124900,1423 S.B.S,가이아나,NaN,NaN,NaN,NaN,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,NaN,NaN
1,11201698,카발란 클래식 싱글몰트,Kavalan Classic Single Malt,싱글몰트,700,40.0,144000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,/m/_next/image?url=https%3A%2F%2Fd1e2y5wc27crn...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...
2,11201699,카발란 콘서트마스터 포트 캐스크 피니시,Kavalan Concertmaster Port Cask Finish,싱글몰트,700,40.0,173000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/sm...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...
3,11201700,카발란 콘서트마스터 포트 캐스크 피니시 1L,Kavalan Concertmaster Port Cask Finish 1L,싱글몰트,1000,40.0,200000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...
4,11201701,카발란 콘서트마스터 셰리 캐스크 피니시,Kavalan Concertmaster Sherry Cask Finish,싱글몰트,700,40.0,204000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...


In [92]:
save_dir = data_dir / 'photos'
os.makedirs(save_dir, exist_ok=True)

def get_extension_from_url(url):
    """Extract extension from URL if possible, else use .jpg"""
    path = urlparse(url).path
    ext = os.path.splitext(path)[1]
    if ext.lower() in [".jpg", ".jpeg", ".png", ".webp"]:
        return ext
    return ".jpg"

def download_image_if_not_exists(url, dest_path):
    """Download a single image safely"""
    try:
        if os.path.exists(dest_path):
            return True
        else:
            resp = requests.get(url, timeout=10)
            resp.raise_for_status()
            with open(dest_path, "wb") as f:
                f.write(resp.content)
            return True
    except Exception as e:
        print(f"❌ Failed {url[:30]}...")
        return False

all_paths = []

# Iterate through each row
for _, row in df.iterrows():
    whisky_id = row["id"]

    ext_url = row.get("external_image_url")
    if not(pd.isna(ext_url) or not isinstance(ext_url, str)):
        ext = get_extension_from_url(ext_url)
        filename = f"{whisky_id}_1{ext}"
        dest_path = os.path.join(save_dir, filename)
        result = download_image_if_not_exists(ext_url, dest_path)
        if result:
            all_paths.append(f"photos/{whisky_id}_{idx}{ext}")
            continue
        
    fl_url = row.get("full_image_url")
    if not(pd.isna(fl_url) or not isinstance(fl_url, str)):
        ext = get_extension_from_url(fl_url)
        filename = f"{whisky_id}_2{ext}"
        dest_path = os.path.join(save_dir, filename)
        result = download_image_if_not_exists(fl_url, dest_path)
        if result:
            all_paths.append(f"photos/{whisky_id}_2{ext}")
            continue

    all_paths.append(float('nan'))
    

df['local_full_path'] = all_paths
df['local_full_path'].head(10)

❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m/_next/image?url=https%3A%2F...
❌ Failed /m

0    photos/63800001_1.webp
1     photos/11201698_2.png
2    photos/11201699_1.webp
3    photos/11201700_1.webp
4    photos/11201701_1.webp
5     photos/11201702_2.png
6    photos/11201703_1.webp
7     photos/11201704_2.png
8     photos/11201705_2.png
9    photos/11201706_1.webp
Name: local_full_path, dtype: object

In [95]:
# Drop rows with NaN in full_image_url
print(f"Before: {len(df)} rows")

df.dropna(subset=["local_full_path"], inplace=True)

print(f"After : {len(df)} rows")

Before: 2958 rows
After : 2943 rows


In [96]:
#better to use english!!
category_map = {
    "싱글몰트": "Single Malt",
    "블렌디드": "Blended",
    "버번": "Bourbon",
    "브랜디": "Brandy",
    "라이": "Rye",
    "기타": "Other"
}

df["eng_category"] = df["category"].map(category_map)

In [97]:
df.head(5)

,id,name,name_eng,category,volume,alcohol,price,brand,nation,bottled,region,region_detail,distillery,importer,external_image_url,image_url,full_image_url,local_full_path,eng_category
0,63800001,1423 S.B.S 가이아나 2020,1423 S.B.S Guyana 2020,기타,700,66.4,124900,1423 S.B.S,가이아나,NaN,NaN,NaN,NaN,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,NaN,NaN,photos/63800001_1.webp,Other
1,11201698,카발란 클래식 싱글몰트,Kavalan Classic Single Malt,싱글몰트,700,40.0,144000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,/m/_next/image?url=https%3A%2F%2Fd1e2y5wc27crn...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...,photos/11201698_2.png,Single Malt
2,11201699,카발란 콘서트마스터 포트 캐스크 피니시,Kavalan Concertmaster Port Cask Finish,싱글몰트,700,40.0,173000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/sm...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...,photos/11201699_1.webp,Single Malt
3,11201700,카발란 콘서트마스터 포트 캐스크 피니시 1L,Kavalan Concertmaster Port Cask Finish 1L,싱글몰트,1000,40.0,200000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...,photos/11201700_1.webp,Single Malt
4,11201701,카발란 콘서트마스터 셰리 캐스크 피니시,Kavalan Concertmaster Sherry Cask Finish,싱글몰트,700,40.0,204000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,https://d1e2y5wc27crnp.cloudfront.net/media/co...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1120...,photos/11201701_1.webp,Single Malt


In [98]:
sliced = df.drop(['image_url', 'full_image_url', 'external_image_url', 'category'], axis=1)

In [100]:
#Save the data just filtered
filtered_csv_url = data_dir / 'whiskies_filtered.csv'

sliced.to_csv(filtered_csv_url, index=False)

In [101]:
sliced.head()

,id,name,name_eng,volume,alcohol,price,brand,nation,bottled,region,region_detail,distillery,importer,local_full_path,eng_category
0,63800001,1423 S.B.S 가이아나 2020,1423 S.B.S Guyana 2020,700,66.4,124900,1423 S.B.S,가이아나,NaN,NaN,NaN,NaN,NaN,photos/63800001_1.webp,Other
1,11201698,카발란 클래식 싱글몰트,Kavalan Classic Single Malt,700,40.0,144000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,photos/11201698_2.png,Single Malt
2,11201699,카발란 콘서트마스터 포트 캐스크 피니시,Kavalan Concertmaster Port Cask Finish,700,40.0,173000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,photos/11201699_1.webp,Single Malt
3,11201700,카발란 콘서트마스터 포트 캐스크 피니시 1L,Kavalan Concertmaster Port Cask Finish 1L,1000,40.0,200000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,photos/11201700_1.webp,Single Malt
4,11201701,카발란 콘서트마스터 셰리 캐스크 피니시,Kavalan Concertmaster Sherry Cask Finish,700,40.0,204000,Kavalan,대만,NaN,NaN,NaN,Kavalan,NaN,photos/11201701_1.webp,Single Malt
